<a href="https://colab.research.google.com/github/wayne-chi/Election_ETL_Pipeline_1/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pymongo[srv]
!pip install load_dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 35.9 MB/s eta 0:00:00


In [7]:


!pip install feather-format


  Preparing metadata (setup.py) ... done
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2436 sha256=521f8ffba91aaabb638460ccfe6f0742d3dd52832d9572c4d6ff19b4103baf8b
  Stored in directory: /root/.cache/pip/wheels/a6/ff/5d/4f10de26fe5ddef243c97f13c6cf579d7353d659e41a05c3a6
Successfully built feather-format


# Import Modules

In [8]:
## import
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import sys
import os

from dotenv import load_dotenv
#

import pprint as pp
import feather
import copy
from geopy.geocoders import ArcGIS
from tqdm import tqdm


load_dotenv()

False

## Read Data

In [15]:


df = pd.read_csv('RIVERS_crosschecked.csv')
df['Address'] = df['PU-Name'] + ', ' + df['Ward'] + ', ' + df['LGA'] + ', ' + df['State']

df.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,Address
0,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-001,"OGBEMA I, TOWN HALL",129,991,True,-1,False,False,False,False,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"OGBEMA I, TOWN HALL, ABUA I, ABUA-ODUAL, RIVERS"
1,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-002,"OGBEMA II, C.P.S HALL",129,553,True,-1,False,True,False,False,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"OGBEMA II, C.P.S HALL, ABUA I, ABUA-ODUAL, RIVERS"
2,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-003,"OGBEMA III, C.P.S HALL",29,494,True,-1,False,True,False,False,UNKNOWN,10,1,17,0,https://docs.inecelectionresults.net/elections...,"OGBEMA III, C.P.S HALL, ABUA I, ABUA-ODUAL, RI..."
3,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-004,"OGBEMA IV, TOWN HALL",89,394,True,-1,False,False,False,False,UNKNOWN,14,5,63,0,https://docs.inecelectionresults.net/elections...,"OGBEMA IV, TOWN HALL, ABUA I, ABUA-ODUAL, RIVERS"
4,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-005,IGUTA ADAUYO TOWN HALL I,46,214,True,-1,False,False,False,False,UNKNOWN,5,0,41,0,https://docs.inecelectionresults.net/elections...,"IGUTA ADAUYO TOWN HALL I, ABUA I, ABUA-ODUAL,..."


In [ ]:
me

In [14]:

geolocator = ArcGIS(timeout=10)
def get_lat_long(address, retries=3, delay=5):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None



In [16]:
tqdm.pandas()
df['Latitude'], df['Longitude'] = zip(*df['Address'].progress_apply(get_lat_long))

100%|██████████| 4769/4769 [36:07<00:00,  2.20it/s]


In [17]:


df.reset_index(drop=True, inplace=True)
feather.write_dataframe(df, 'rivers_cleaned.feather')


In [18]:
df.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,Address,Latitude,Longitude
0,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-001,"OGBEMA I, TOWN HALL",129,991,True,-1,False,...,False,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"OGBEMA I, TOWN HALL, ABUA I, ABUA-ODUAL, RIVERS",4.81285,6.60239
1,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-002,"OGBEMA II, C.P.S HALL",129,553,True,-1,False,...,False,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"OGBEMA II, C.P.S HALL, ABUA I, ABUA-ODUAL, RIVERS",4.81285,6.60239
2,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-003,"OGBEMA III, C.P.S HALL",29,494,True,-1,False,...,False,UNKNOWN,10,1,17,0,https://docs.inecelectionresults.net/elections...,"OGBEMA III, C.P.S HALL, ABUA I, ABUA-ODUAL, RI...",4.81285,6.60239
3,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-004,"OGBEMA IV, TOWN HALL",89,394,True,-1,False,...,False,UNKNOWN,14,5,63,0,https://docs.inecelectionresults.net/elections...,"OGBEMA IV, TOWN HALL, ABUA I, ABUA-ODUAL, RIVERS",4.81285,6.60239
4,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-005,IGUTA ADAUYO TOWN HALL I,46,214,True,-1,False,...,False,UNKNOWN,5,0,41,0,https://docs.inecelectionresults.net/elections...,"IGUTA ADAUYO TOWN HALL I, ABUA I, ABUA-ODUAL,...",4.85763,6.64519


In [20]:

df = feather.read_dataframe('rivers_cleaned.feather')

def convert_to_geometry(row):
  return {
    "type": "Point",
    "coordinates": [row['Longitude'], row['Latitude']]
  }

df['location'] = df.apply(convert_to_geometry, axis=1)


In [25]:


# Get MongoDB connection string from environment variable
MONGO_URI = os.getenv("MONGODB_de_URI")

# Create a MongoClient object
client = MongoClient(MONGO_URI)

# Print confirmation message
print("MongoClient successfully initiated.")


MongoClient successfully initiated.


In [29]:
df.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,...,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,Address,Latitude,Longitude,location
0,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-001,"OGBEMA I, TOWN HALL",129,991,True,-1,False,...,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"OGBEMA I, TOWN HALL, ABUA I, ABUA-ODUAL, RIVERS",4.81285,6.60239,"{'type': 'Point', 'coordinates': [6.60239, 4.8..."
1,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-002,"OGBEMA II, C.P.S HALL",129,553,True,-1,False,...,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...,"OGBEMA II, C.P.S HALL, ABUA I, ABUA-ODUAL, RIVERS",4.81285,6.60239,"{'type': 'Point', 'coordinates': [6.60239, 4.8..."
2,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-003,"OGBEMA III, C.P.S HALL",29,494,True,-1,False,...,UNKNOWN,10,1,17,0,https://docs.inecelectionresults.net/elections...,"OGBEMA III, C.P.S HALL, ABUA I, ABUA-ODUAL, RI...",4.81285,6.60239,"{'type': 'Point', 'coordinates': [6.60239, 4.8..."
3,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-004,"OGBEMA IV, TOWN HALL",89,394,True,-1,False,...,UNKNOWN,14,5,63,0,https://docs.inecelectionresults.net/elections...,"OGBEMA IV, TOWN HALL, ABUA I, ABUA-ODUAL, RIVERS",4.81285,6.60239,"{'type': 'Point', 'coordinates': [6.60239, 4.8..."
4,RIVERS,ABUA-ODUAL,ABUA I,32-01-01-005,IGUTA ADAUYO TOWN HALL I,46,214,True,-1,False,...,UNKNOWN,5,0,41,0,https://docs.inecelectionresults.net/elections...,"IGUTA ADAUYO TOWN HALL I, ABUA I, ABUA-ODUAL,...",4.85763,6.64519,"{'type': 'Point', 'coordinates': [6.64519, 4.8..."


In [33]:

df_filtered = df[~df['Latitude'].isnull() & ~df['Longitude'].isnull()]
df_filtered = df_filtered.drop(columns=['Latitude', 'Longitude'])



# Access the database and collection
db = client['election']
collection = db['poolingUnits']

# Insert filtered data into MongoDB
collection.insert_many(df_filtered.to_dict('records'))

# Print confirmation message
print("Data successfully inserted into MongoDB.")


Data successfully inserted into MongoDB.


       State         LGA         Ward       PU-Code  \
0     RIVERS  ABUA-ODUAL       ABUA I  32-01-01-001   
1     RIVERS  ABUA-ODUAL       ABUA I  32-01-01-002   
2     RIVERS  ABUA-ODUAL       ABUA I  32-01-01-003   
3     RIVERS  ABUA-ODUAL       ABUA I  32-01-01-004   
4     RIVERS  ABUA-ODUAL       ABUA I  32-01-01-005   
...      ...         ...          ...           ...   
4764  RIVERS         TAI  OYIGBO WEST  32-23-08-009   
4765  RIVERS         TAI  OYIGBO WEST  32-23-08-011   
4766  RIVERS         TAI  OYIGBO WEST  32-23-08-012   
4767  RIVERS         TAI  OYIGBO WEST  32-23-08-013   
4768  RIVERS         TAI  KOROMA/HORO  32-23-04-008   

                                                PU-Name  Accredited_Voters  \
0                                   OGBEMA I, TOWN HALL                129   
1                                 OGBEMA II, C.P.S HALL                129   
2                                OGBEMA III, C.P.S HALL                 29   
3                          

In [35]:

df_filtered_2 = df[df['Latitude'].isnull() & df['Longitude'].isnull()]
df_filtered_2 = df_filtered_2.drop(columns=['Latitude', 'Longitude', 'location'])


l = (len(df_filtered_2))
# Insert filtered data into MongoDB
if l != 0:
  collection.insert_many(df_filtered_2.to_dict('records'))

# Print confirmation message
print("Data successfully inserted into MongoDB.")


Data successfully inserted into MongoDB.


In [ ]:
# Create index on location field
collection.create_index({"location": "2dsphere"})

# Print confirmation message
print("Index successfully created on location field.")
